# Imports für das Notebook

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
from sklearn.metrics import recall_score

# Vorbereitung der Daten

In [32]:
# Einlesen der gesäuberten Daten für Exploration und Visualisierung
heart_data = pd.read_csv("../../resources/data_clean/heart_2020_clean.csv")

# Filtern von zu hohen Schlafzeiten
heart_data = heart_data[heart_data["SleepTime"] <= np.percentile(heart_data["SleepTime"], 99.9)]

# Trainieren des XGBoost-Modells
model_xgb = xgb.XGBClassifier()

# Konvertierung der kategorialen Attribute
labelencoder = LabelEncoder()

heart_data["AgeCategory"] = labelencoder.fit_transform(heart_data["AgeCategory"])
heart_data["KidneyDisease"] = heart_data["KidneyDisease"].map({"Ja": 1, "Nein": 0})
heart_data["Stroke"] = heart_data["Stroke"].map({"Ja": 1, "Nein": 0})
heart_data["GenHealth"] = labelencoder.fit_transform(heart_data["GenHealth"])

# Standardisierung der numerischen Attribute
scaler = StandardScaler()
heart_data["SleepTime"] = scaler.fit_transform(heart_data["SleepTime"].values.reshape(-1, 1))


Die CSV-Datei wird hier so angepasst, dass der nächste Abschnitt sich um das Modell für die Vorhersage kümmern kann.  
Es werden hier die nichtaussagekräftigen Werte aus dem Datensatz gefiltert und kategoriellen Variablen werden umcodiert, sodass sie vom Modell verarbeitet werden können

# XGBoost-Modell

In [35]:
# Trennen von Trainings- und Testdatensatz
X = heart_data[["AgeCategory", "SleepTime", "KidneyDisease", "Stroke", "GenHealth"]]
y = heart_data["HeartDisease"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Weitere Kategorisierung
y_train_encoded = labelencoder.fit_transform(y_train)
y_test_encoded = labelencoder.fit_transform(y_test)

# Training des Modells
model_xgb.fit(X_train, y_train_encoded)

# Vorhersagen auf dem Testdatensatz
y_pred_xgb = model_xgb.predict(X_test)

# Bewertung der Modellleistung anhand des Recalls
recall_xgb = recall_score(y_test_encoded, y_pred_xgb, pos_label=1)

print("XGBoost Recall (Ja):", recall_xgb)

# Speicherung des Models, für einfachere Wiederverwendung ohne erneutes Training
model_xgb.save_model("../../resources/xgb_model.json")
print("XGB-Modell wurde in dem Ordner 'resources' gespeichert")

# Vorhersagen und Wahrscheinlichkeiten auf dem Testdatensatz
# #y_pred_xgb_proba = model_xgb.predict_proba(X_test)

# Extrahieren Sie die Wahrscheinlichkeit für "Herzerkrankt"
# #prob_herz_krank = y_pred_xgb_proba[:, 1]  # Zweite Spalte enthält die Wahrscheinlichkeit für Klasse 1 ("Herzerkrankt")

# Drucken Sie die Vorhersagen mit Wahrscheinlichkeiten
# #for i, prob in enumerate(prob_herz_krank):
    # #print(f"Probe {i+1}: {y_test.iloc[i]} ({prob:.2f}%)")


XGBoost Recall (Ja): 0.9958928724223496
XGB-Modell wurde in dem Ordner 'resources' gespeichert
Probe 1: Nein (0.83%)
Probe 2: Nein (1.00%)
Probe 3: Nein (0.96%)
Probe 4: Nein (1.00%)
Probe 5: Nein (0.61%)
Probe 6: Nein (0.84%)
Probe 7: Ja (0.73%)
Probe 8: Nein (0.97%)
Probe 9: Nein (0.79%)
Probe 10: Nein (0.92%)
Probe 11: Nein (0.97%)
Probe 12: Nein (0.98%)
Probe 13: Nein (0.90%)
Probe 14: Nein (0.92%)
Probe 15: Nein (0.99%)
Probe 16: Nein (0.96%)
Probe 17: Ja (0.80%)
Probe 18: Ja (0.46%)
Probe 19: Nein (0.96%)
Probe 20: Nein (0.99%)
Probe 21: Nein (0.94%)
Probe 22: Nein (0.70%)
Probe 23: Nein (1.00%)
Probe 24: Nein (1.00%)
Probe 25: Nein (0.90%)
Probe 26: Nein (0.91%)
Probe 27: Ja (0.78%)
Probe 28: Nein (0.63%)
Probe 29: Nein (0.99%)
Probe 30: Ja (0.68%)
Probe 31: Nein (1.00%)
Probe 32: Nein (0.96%)
Probe 33: Nein (1.00%)
Probe 34: Nein (0.98%)
Probe 35: Nein (0.78%)
Probe 36: Nein (0.94%)
Probe 37: Nein (0.94%)
Probe 38: Nein (0.99%)
Probe 39: Nein (0.97%)
Probe 40: Nein (0.57%)
Prob

Unsere Modelle basieren auf den 5 wichtigsten Attributen, die auf eine Herzerkrankung hinweisen können. Diese wurden im anderen Notebook in diesem Ordner manuell identifiziert.  
Der Recall, der misst, wie viele der tatsächlich Erkrankten vom Modell korrekt identifiziert werden, liegt in unserem Fall bei herausragenden 99,6%. Dies bedeutet, dass die meisten Vorhersagen des Modells korrekt eine Herzerkrankung erkennen.